## Bash Programming Interactive lecture


## Reminder: Combining bash commands: Pipes

<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f6/Pipeline.svg/560px-Pipeline.svg.png
" style="width: 500px;"/>
</center>

The bash pipe `|` connects `STDOUT` of one command to `STDIN` of another. 

## Live programming: Oslo's weather forecast.

<font color='red'>⚠️ </font>  Unix commands used here:
* **curl**: downloads a website
* **grep**: filter lines with a given keyword 
* **head**: limit the output to the first X lines
* **cut**:  select a specific column defined by a deliminter
* **cowsay**: let a cow say something



### Interactive programming

Solve the following exercises:

1) Implement a weather forcast script the outputs the weather in Bergen.

2) (Optional) Implement a script that outputs headlines of nrk.no.


**Time: 10 minutes**





## More pipe examples

* Sort all files in a directory 
tree, with the largest files appearing first, and
equip the output with paging functionality:

```bash
du -a lectures | sort -rn | less
```

<font color='red'>⚠️ </font> **du** shows the disk usages of files
<br>
<font color='red'>⚠️ </font> **sorts** sorts the input
<br>
<font color='red'>⚠️ </font> **less** paginates the output for easier reading

## More pipe examples

* Save output to disk and display it on the console:

```bash
./compile_model.sh | tee compile.log
```

<font color='red'>⚠️ </font> **tee** writes `STDIN` to `STDOUT` and file
<img src="figs/tee.png" style="width: 500px;"/>

## Quiz: Can you simplify this code?

```bash
ls > files.txt ; grep 2017 < files.txt ; rm files.txt
```

<font color='red'>⚠️ </font> **CMD1 ; CMD2** executes `CMD2` after `CMD1`.

The code above implements an (inefficient) pipe. 

Better:
```bash
ls | grep 2017
```

## Functions

Bash has simple support for functions.

```bash
function geturl {
    
    # function arguments: $1 $2 $3 and so on
    
	if [ $1 == "Oslo" ]; then
		url="https://www.yr.no/place/Norway/Oslo/Oslo/Oslo/"
	fi
}


# call:
geturl "Oslo"
echo $url       # Bash functions have no return values, but all variables are global
```

### Interactive programming


Implement a "geturl" function that takes a city as input and stores the assoicated yr url as output. 

The function should work for Bergen, Oslo and Stavanger. 

If an invalid city is provided, the function should exit the script and print an error message.
 
**Time: 10 minutes**





## Convenient debugging tool: -x


Each source code line is printed prior to its execution if you add -x as option to `/bin/sh` or `/bin/bash`

Either in the header

```bash
#!/bin/bash -x
```
or on the command line:
```bash
bash -x scripts/backup.sh
```

**Live example**

In [ ]:
!scripts/backup.sh

## Reminder: command line arguments

Bash provides the special (array) variable `$` for accessing these arguments:

* `$0` contains the script name
* `$1` contains the first command line option
* `$2` contains the second command line option
* ...

### Interactive programming


1) Extend your weather forecast script so that it takes the city as a command line option: 

Example usage:
```bash
weather.sh Oslo   # Prints out Oslo's weather
```

```bash
weather.sh Mars   # Prints out an error message (not a city)
```

2) (Optional): Print a short usage documentation for the script when no command line options are provided.

 
**Time: 10 minutes**





### Running an application in the foreground

**Syntax**: 

* `cmd &` executes a program in the background. 
* Use `wait` to block a script until all background processes are completed.

**Example**:
```bash
myprog -c file.1 -p -f -q < my_input_file &

# any commands here will be executed while myprog is still running
```

**Live demo**:
`scripts/background_test.sh`

## File globbing, for loop on the command line


List all .ps and .gif files using wildcard notation:

```bash
files=$(ls *.ps *.gif)

# compress and move the files:
gzip $files
for file in $files; do
  mv ${file}.gz $HOME/images
```



<font color='red'>⚠️ </font> **gzip** compresses a file,
**mv** moves a file, **$HOME** is a string with the path to your home directory.

## The find command

Very useful command!


<font color='red'>⚠️ </font> `find` visits all files in a directory tree and can execute one or more commands for every file

Basic example: find the `oscillator` codes

```bash
find $scripting/src -name 'oscillator*' -print
```

Or find all log and PDF files

```bash
find $HOME \( -name '*.log' -o -name '*.pdf' \) -print
```

# That is it for today

<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a4/Anatomy_of_a_Sunset-2.jpg/2880px-Anatomy_of_a_Sunset-2.jpg" style="width: 500px;"/>
</center>

## Applications of find (1)


Find all files larger than 2000 blocks a 512 bytes (=1Mb):

```bash
find $HOME -name '*' -type f -size +2000 -exec ls -s {} \;
```

Remove all these files:

```bash
find $HOME -name '*' -type f -size +2000 \
           -exec ls -s {} \; -exec rm -f {} \;
```

or ask the user for permission to remove:

```bash
find $HOME -name '*' -type f -size +2000 \
           -exec ls -s {} \; -ok rm -f {} \;
```

### Backup slides

## Redirects and pipes can be combined

View `STDOUT` and `STDERR` with paging functionality:

```bash   
./compile 2>&1 | tee compile.log | less 
```

## Example: bundle files


Pack a series of files into one file

Executing this single file as a Bash script packs out all the individual files again

Usage:

```bash
bundle file1 file2 file3 > onefile  # pack
bash onefile # unpack
```

Writing `bundle` is easy:

```bash
#/bin/sh
for i in $@; do
    echo "echo unpacking file $i"
    echo "cat > $i <<EOF"
    cat $i
    echo "EOF"
done
```

## The bundle output file


Consider 2 fake files: `file1`:

```bash
Hello, World!
No sine computations today
```

and `file2`:

```bash
1.0 2.0 4.0
0.1 0.2 0.4
```

Running `bundle file1 file2` yields the output

```bash
echo unpacking file file1
cat > file1 <<EOF
Hello, World!
No sine computations today
EOF
echo unpacking file file2
cat > file2 <<EOF
1.0 2.0 4.0
0.1 0.2 0.4
EOF
```

**Note**: In the terminal, you can send a foreground job into the background by pressing `Ctrl-Z` followed by the `bg` command, and retrieve it again with `fg`.

How to return a value from a function? Define a new variable within the function - all functions are global!

## Applications of find (2)


Find all files not being accessed for the last 90 days:

```bash
find $HOME -name '*' -atime +90 -print
```

and move these to /tmp/trash:

```bash
find $HOME -name '*' -atime +90 -print \
           -exec mv -f {} /tmp/trash \;
```

## Tar and gzip


<font color='red'>⚠️ </font> The `tar` command can pack single files or  all files in a directory tree into one file, which can be unpacked later

```bash
tar -cvf myfiles.tar mytree file1 file2

# options:
# c: pack, v: list name of files, f: pack into file

# unpack the mytree tree and the files file1 and file2:
tar -xvf myfiles.tar

# options:
# x: extract (unpack)
```

The tarfile can be compressed:

```bash
gzip mytar.tar
# result: mytar.tar.gz
```

## A find/tar/gzip example


Pack all PostScript figures:

```bash
tar -cvf ps.tar `find $HOME -name '*.ps' -print`
gzip ps.tar
```